In [1]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd
import pandas as pd
import sys
from ydata_profiling import ProfileReport
sys.path.append('./../../')
from src.data.conn_oracle import connect_oracle_trusted, connect_oracle_refined
from src.data.data_quality_tarefcon import clean_tarefcon
from src.data.data_clean import clean_clientes,clean_facas,clean_itens,clean_maquina,clean_paradas,clean_pedidos,clean_tarefcon
load_dotenv()

server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")

conn = pyodbc.connect(
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)




### Load Data

#### 01.Tarefcon

In [2]:

df_tarefcon = pd.read_sql("""
 SELECT
    Maquina,
    FlagParada,
    CodigoParadaOuConv,
    Turma,
    OP,
    Pedido,
    Item,
    Reprogramacao,
    Arranjo,
    Gramatura,
    QuantidadeProgramada,
    ChapasAlimentadas,
    QuantidadeProduzida,
    QuantidadeAjuste,
    DuracaoPrevista,
    Inicio,
    Fim,
    DiaDaTurma,
    IDCliente,
    Usuario,
    OrigemRegistro,
    DescOrigemRegistro,
    SkipFeed,
    OPOndulada,
    Duracao,
    Faca
    FROM TAREFCON
    WHERE DIADATURMA >= '2022-01-01';
""", conn)

df_tarefcon.head()

/tmp/ipykernel_2714859/1869721092.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tarefcon = pd.read_sql("""


,Maquina,FlagParada,CodigoParadaOuConv,Turma,OP,Pedido,Item,Reprogramacao,Arranjo,Gramatura,...,Fim,DiaDaTurma,IDCliente,Usuario,OrigemRegistro,DescOrigemRegistro,SkipFeed,OPOndulada,Duracao,Faca
0,ACB,0,NaN,B,587252-1/804302,587252-1,804302,1.0,1.0,376.0,...,2022-01-04 21:00:00.000,2022-01-04,8399.0,191.0,0.0,Manual padrão,0,None,480,None
1,ACB,0,NaN,C,585577-4/788942,585577-4,788942,1.0,1.0,369.0,...,2022-01-04 22:00:00.000,2022-01-04,11572.0,429.0,0.0,Manual padrão,0,PRD046120/788942,60,None
2,ACB,1,1.0,C,587252-1/804302,None,None,NaN,NaN,NaN,...,2022-01-04 21:10:05.990,2022-01-04,NaN,429.0,0.0,Manual padrão,0,None,10,None
3,ACB,0,NaN,C,587252-1/804302,587252-1,804302,1.0,1.0,376.0,...,2022-01-05 03:15:00.000,2022-01-04,8399.0,429.0,0.0,Manual padrão,0,None,375,None
4,ACB,1,1.0,C,587252-1/804302,None,None,NaN,NaN,NaN,...,2022-01-04 21:10:05.990,2022-01-04,NaN,429.0,0.0,Manual padrão,0,None,10,None


In [3]:

df_tarefcon = df_tarefcon.drop_duplicates().reset_index(drop=True)
df_tarefcon = clean_tarefcon(df_tarefcon)
df_tarefcon = df_tarefcon[sorted(df_tarefcon.columns)]
df_tarefcon.to_parquet('./../../data/raw/tb_tarefcon.parquet')

### 02.pedidos

In [4]:
df_pedidos = pd.read_sql("""
    SELECT
        Pedido,
        Item,
        StatusPedido,
        DescrStatusPedido,
        Suspenso,
        SuspOuCancel,
        IDCliente,
        Referencia,
        CodigoReferencia,
        TipoABNT,
        idTipoFT2,
        ExigeLaudo,
        Gramatura,
        ColunaMinimo,
        CobbIntMaximo,
        Compressao,
        Chapa,
        Composicao,
        Largura,
        RefiloLargura,
        Comprimento,
        RefiloComprimento,
        MultLarg,
        MultComp,
        Arranjo,
        RefugoCliente,
        PesoPeca,
        VincoLarg1,
        VincoLarg2,
        VincoLarg3,
        VincoComp1,
        VincoComp2,
        VincoComp3,
        VincoComp4,
        VincoComp5,
        Lap,
        ProlongLap,
        LapNoComp,
        LapInterno,
        Refilado,
        ResinaInterna,
        Amarrado,
        Paletizado,
        PacotesLargura,
        PacotesComprimento,
        PacotesAltura,
        PecasPorPacote,
        PecasPorPalete,
        UnidadesPorPalete,
        PacotesPorPalete,
        Espelho,
        Filme,
        Faca,
        Cor1,
        ConsumoCor1,
        Cor2,
        ConsumoCor2,
        Cor3,
        ConsumoCor3,
        Cor4,
        ConsumoCor4,
        NrCores,
        DataEntrega2,
        QuantidadePedida,
        QuantidadePedidaMin,
        QuantidadePedidaMax,
        TipoEntrega,
        DescTipoEntrega,
        LarguraInterna,
        ComprimentoInterno,
        AlturaInterna,
        LargPeca,
        CompPeca,
        DataEntregaOriginal,
        DescrTipoDoPedido,
        CompPacote,
        LargPacote,
        AlturaPacote,
        CompPaleteFechado,
        LargPaleteFechado,
        AlturaPaleteFechado,
        PesoCaixa,
        PathFiguraDoLastro,
        IDPalete,
        AreaBrutaPecaComRefilos,
        AreaBrutaPeca,
        AreaLiquidaPeca,
        AreaBrutaChapa,
        AreaLiquidaChapa,
        VolumePaleteFechadoM3,
        VolumePacoteFechadoM3,
        VolumeFechadoPedido
    FROM PEDIDOS
    WHERE DataEntregaOriginal >= '2022-01-01';
""", conn)

df_pedidos = df_pedidos.drop_duplicates().reset_index(drop=True)

df_pedidos = df_pedidos[sorted(df_pedidos.columns)]
df_pedidos = clean_pedidos(df_pedidos)
df_pedidos.to_parquet("./../../data/raw/tb_pedidos.parquet")
df_pedidos.head()

/tmp/ipykernel_2714859/3466741076.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pedidos = pd.read_sql("""


,CD_PEDIDO,CD_ITEM,ID_CLIENTE,CD_PALETE,CD_TIPOFT2,CD_REFERENCIA,DT_ENTREGA2,DT_ENTREGAORIGINAL,FL_AMARRADO,FL_CHAPA,...,QT_CONSUMOCOR4,VL_VINCOLARG1,VL_VINCOLARG2,VL_VINCOLARG3,VL_VINCOCOMP1,VL_VINCOCOMP2,VL_VINCOCOMP3,VL_VINCOCOMP4,VL_VINCOCOMP5,VL_LAP
0,574075-1,719410,12302,70,-1,19827,2022-03-14,2022-03-10,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,574075-2,719410,12302,70,-1,19827,2022-01-03,2022-01-03,1,0,...,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,578237-2,760980,10555,8,-1,448898,2022-01-13,2022-01-13,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,578237-3,760980,10555,8,-1,448898,2022-01-17,2022-01-14,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,579947-10,628840,11833,50,-1,220700,2022-01-04,2022-01-04,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_pedidos = df_pedidos[sorted(df_pedidos.columns)]
df_pedidos = clean_pedidos(df_pedidos)
df_pedidos.to_parquet('./../../data/raw/tb_pedidos.parquet')
df_pedidos.head()

,CD_PEDIDO,CD_ITEM,ID_CLIENTE,CD_PALETE,CD_TIPOFT2,CD_REFERENCIA,DT_ENTREGA2,DT_ENTREGAORIGINAL,FL_AMARRADO,FL_CHAPA,...,VL_REFUGOCLIENTE,VL_PESOPECA,VL_PESOCAIXA,VL_AREABRUTAPECA,VL_AREALIQUIDAPECA,VL_AREABRUTACHAPA,VL_AREALIQUIDACHAPA,VL_VOLUMEFECHADOPEDIDO,VL_VOLUMEPACOTEFECHADOM3,VL_VOLUMEPALETEFECHADOM3
0,574075-1,719410,12302,70,-1,19827,2022-03-14,2022-03-10,1,0,...,2.7,0.0,175.0,0.404079,0.393169,1.243172,1.212237,2.87164,0.029094,2.87164
1,574075-2,719410,12302,70,-1,19827,2022-01-03,2022-01-03,1,0,...,2.7,0.0,175.0,0.404079,0.393169,1.243172,1.212237,2.87164,0.029094,2.87164
2,578237-2,760980,10555,8,-1,448898,2022-01-13,2022-01-13,1,0,...,0.8,0.0,188.0,0.405750,0.402504,1.673980,1.623000,17.51200,0.060863,1.75120
3,578237-3,760980,10555,8,-1,448898,2022-01-17,2022-01-14,1,0,...,0.8,0.0,188.0,0.405750,0.402504,1.673980,1.623000,17.51200,0.060863,1.75120
4,579947-10,628840,11833,50,-1,220700,2022-01-04,2022-01-04,0,0,...,3.2,176.0,177.0,0.410366,0.397234,1.696071,1.641464,14.30604,0.061555,2.04372


### 03.Itens

In [6]:
df_itens = pd.read_sql("""
    SELECT
        Item,
        idTipoFT2,
        IDFamilia,
        EstadoFT_Detec,
        TextoEstadoFT_Detec,
        StatusFT,
        TextoStatusFT,
        IDCliente,
        Referencia,
        CodigoReferencia,
        TipoABNT,
        ExigeLaudo,
        Gramatura,
        ColunaMinimo,
        CobbIntMaximo,
        Compressao,
        Composicao,
        Largura,
        RefiloLargura,
        Comprimento,
        RefiloComprimento,
        MultLarg,
        MultComp,
        Arranjo,
        RefugoCliente,
        VincoLarg1,
        VincoLarg2,
        VincoLarg3,
        VincoComp1,
        VincoComp2,
        VincoComp3,
        VincoComp4,
        VincoComp5,
        Lap,
        ProlongLap,
        LapNoComp,
        LapInterno,
        Refilado,
        ResinaInterna,
        Amarrado,
        Paletizado,
        PacotesLargura,
        PacotesComprimento,
        PacotesAltura,
        PecasPorPacote,
        PecasPorPalete,
        PacotesPorPalete,
        UnidadesPorPalete,
        Espelho,
        Filme,
        Faca,
        Cor1,
        ConsumoCor1,
        Cor2,
        ConsumoCor2,
        Cor3,
        ConsumoCor3,
        Cor4,
        ConsumoCor4,
        NrCores,
        LarguraInterna,
        ComprimentoInterno,
        AlturaInterna,
        LargPeca,
        CompPeca,
        CompPacote,
        LargPacote,
        AlturaPacote,
        CompPaleteFechado,
        LargPaleteFechado,
        AlturaPaleteFechado,
        PesoCaixa,
        PathFiguraDoLastro,
        IDPalete,
        AreaBrutaPecaComRefilos,
        AreaBrutaPeca,
        AreaLiquidaPeca,
        AreaBrutaChapa,
        AreaLiquidaChapa,
        VolumePaleteFechadoM3,
        VolumePacoteFechadoM3
    FROM ITENS;
""", con= conn)

/tmp/ipykernel_3298639/2930505811.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_itens = pd.read_sql("""


In [7]:

df_itens = clean_itens(df_itens)
df_itens = df_itens[sorted(df_itens.columns)]
df_itens.to_parquet('./../../data/raw/tb_itens.parquet')
df_itens.head()

,CD_ESPELHO,CD_FACA,CD_FAMILIA,CD_FILME,CD_ITEM,CD_PALETE,CD_REFERENCIA,CD_TIPOFT2,FL_AMARRADO,FL_EXIGELAUDO,...,VL_VINCOCOMP1,VL_VINCOCOMP2,VL_VINCOCOMP3,VL_VINCOCOMP4,VL_VINCOCOMP5,VL_VINCOLARG1,VL_VINCOLARG2,VL_VINCOLARG3,VL_VOLUMEPACOTEFECHADOM3,VL_VOLUMEPALETEFECHADOM3
0,0,-1,51,0,00001,-1,,-1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1,-1,9,1,000020,5,935476,-1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060449,1.147500
2,1,-1,9,1,000030,1,935476,-1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048995,1.355319
3,1,-1,9,1,000040,42,935476,-1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033770,1.944000
4,1,-1,9,0,000042,7,,-1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005526,1.188000


### 04.Paradas

In [8]:
df_paradas = pd.read_sql("""
    SELECT
        Parada,
        Descricao,
        UsadaConversao,
        Desativada,
        flagExterna
    FROM PARADAS; """, conn)

/tmp/ipykernel_3298639/3270135405.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_paradas = pd.read_sql("""


In [9]:

df_paradas = df_paradas.drop_duplicates().reset_index(drop=True)
df_paradas = clean_paradas(df_paradas)
df_paradas = df_paradas[sorted(df_paradas.columns)]

df_paradas.to_parquet('./../../data/raw/tb_paradas.parquet')


### 05. Maquina

In [10]:
df_maquina = pd.read_sql("""                    
    SELECT
        Maquina,
        Tipo,
        NrDeCores,
        IDGrupoMaquina
    FROM MAQUINA;
""", conn)

/tmp/ipykernel_3298639/3626826674.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_maquina = pd.read_sql("""


In [11]:

df_maquina = df_maquina.drop_duplicates().reset_index(drop=True)
df_maquina = clean_maquina(df_maquina)
df_maquina = df_maquina[sorted(df_maquina.columns)]

df_maquina.to_parquet('./../../data/raw/tb_maquina.parquet')


### 06. Facas

In [12]:
df_facas = pd.read_sql("""                    
SELECT
    CodFaca,
    Status,
    CompLamina,
    DesativadoSN
FROM FACAS;
""", conn)

/tmp/ipykernel_3298639/145594070.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_facas = pd.read_sql("""


In [13]:

df_facas = df_facas.drop_duplicates().reset_index(drop=True)
df_facas = clean_facas(df_facas)
df_facas = df_facas[sorted(df_facas.columns)]

df_facas.to_parquet('./../../data/raw/tb_facas.parquet')


### 07. Clientes

In [14]:
df_clientes = pd.read_sql(""" 
SELECT
IDCliente,                                 
Cliente,                                   
CodCliente,                                
CodRepresentante                                  
FROM Clientes;
""", conn)



df_clientes = df_clientes.drop_duplicates().reset_index(drop=True)
df_clientes = clean_clientes(df_clientes)
df_clientes = df_clientes[sorted(df_clientes.columns)]

df_clientes.to_parquet('./../../data/raw/tb_clientes.parquet')



/tmp/ipykernel_3298639/247591813.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clientes = pd.read_sql("""
